# Resilience
Calculate metrics showing the resilience of wikipedians.
I came up only with the mean daily edit numbers per user for now that I calculated and saved in this notebook.

In [1]:
import pandas as pd
import numpy as np

In [2]:
DATA_PATH = "/dlabdata1/turkish_wiki"

In [3]:
registered_edits = pd.read_csv(f"{DATA_PATH}/processed_data/account_edits.csv", index_col = 0)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
registered_edits['event_timestamp'] = pd.to_datetime(registered_edits['event_timestamp'],   utc = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
block_dates = [pd.to_datetime('2017-04-29', utc = True), pd.to_datetime('2020-01-15', utc = True)]

In [15]:
registered_edits['period'] = np.nan
registered_edits.loc[registered_edits['event_timestamp'] < block_dates[0], 'period'] = 'pre-block'
registered_edits.loc[(registered_edits['event_timestamp'] >= block_dates[0]) & (registered_edits['event_timestamp'] < block_dates[1]) , 'period'] = 'block'
registered_edits.loc[registered_edits['event_timestamp'] >= block_dates[1], 'period'] = 'after-block'

## Mean daily edits per user.
Shows the number of edits divided by number of active users for a given day.

In [22]:
group = registered_edits.groupby(['period', pd.Grouper(key = 'event_timestamp', freq='D')])

In [24]:
mean_daily_edits = group.size()/group.event_user_id.nunique()

In [31]:
mean_daily_edits = mean_daily_edits.reset_index()

In [33]:
mean_daily_edits.columns = ['period', 'date', 'avg_number_of_edits_per_active_user']

In [8]:
mean_daily_edits.to_csv(f"{DATA_PATH}/processed_data/daily_avg_edits.csv", index = False)